In [38]:
import psycopg2
import boto3
import pandas as pd
import yfinance as yf
import psycopg2.extras as extras

In [39]:
def connect_to_db() -> object:
    """
    Fetch DB username and password from AWS SSM and connect to RDS database.
    """
    aws_client = boto3.client("ssm")

    parameters = aws_client.get_parameters(
        Names=[
            "stock_twitter_db_user",
            "stock_twitter_db_password",
        ],
        WithDecryption=True,
    )

    credentials = {}
    for parameter in parameters["Parameters"]:
        credentials[parameter["Name"]] = parameter["Value"]

    db_host = "stock-twitter-db.cdx9enjjuwaj.us-east-1.rds.amazonaws.com"
    db_name = "postgres"
    db_port = "5432"
    db_user = credentials["stock_twitter_db_user"]
    db_pass = credentials["stock_twitter_db_password"]
    conn = psycopg2.connect(
        host=db_host, database=db_name, user=db_user, password=db_pass, port=db_port
    )

    return conn

In [40]:
conn = connect_to_db()

In [7]:
pd.read_sql("SELECT * FROM stock", con=conn)

c:\Users\Luca\Documents\Coding\Shopify-Project\venv\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,ticker,name
0,URA,Uranium ETF


In [42]:
pd.read_sql("SELECT * FROM tweet", con=conn)

c:\Users\Luca\Documents\Coding\Shopify-Project\venv\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,twitter_id,username,text,created_at,sentiment
0,1,1532787377906896901,LeclercT40,@PetrNovikov12 @DeepFakeFuori @Osinttechnical ...,2022-06-03,neutral
1,2,1532787214123294725,TNT,RT @Vishal_alt: @zerohedge US itself buys Russ...,2022-06-03,neutral
2,3,1532787137388728322,📐triANGLE Investor,@quakes99 how are we standing with #uranium sp...,2022-06-03,neutral
3,4,1532786888825655299,Jane Busby,@relllattable @astro1095 @AlvaroSilva1906 i me...,2022-06-03,negative
4,5,1532786819909156866,Itallstartshere,United States is funding Russia’s war by buyin...,2022-06-03,neutral
...,...,...,...,...,...,...
2395,2396,1532496704196562949,Ryan Roloff,RT @Antiwarcom: Former IAEA Inspector Wrongly ...,2022-06-02,negative
2396,2397,1532496499799883776,Ironside Capital,RT @KoporeMetals: We are pleased to provide a ...,2022-06-02,positive
2397,2398,1532496443784966146,Ironside Capital,RT @KoporeMetals: Kopore Metals has posted an ...,2022-06-02,neutral
2398,2399,1532496399610560512,▫▫▭▭▯amano▯▱▱（低浮上）,RT @MayRatio: Today is my birthday\n\nMy best ...,2022-06-02,positive


In [16]:
tweets = [{'twitter_id': 1532674306383159298, 'username': 'Luciano Pestarino', 'text': 'RT @giginator_: #Uranium names ripped overnight in Nth America\n$CCJ +7%\n$DNN +5%\n$URG +14%\n$UUUU +10%\nCantor Fitz reckons they picked the b…', 'created_at': '20220603-104325', 'sentiment': 'neutral'},{'twitter_id': 1532674252956123136, 'username': 'ArabiskaPartiet', 'text': '"It’s not just that the 4 billion tons of uranium in seawater now would fuel a thousand 1000-MW nuclear power plant… https://t.co/WO4uvlNwy8', 'created_at': '20220603-104312', 'sentiment': 'neutral'}]
tweets

[{'twitter_id': 1532674306383159298,
  'username': 'Luciano Pestarino',
  'text': 'RT @giginator_: #Uranium names ripped overnight in Nth America\n$CCJ +7%\n$DNN +5%\n$URG +14%\n$UUUU +10%\nCantor Fitz reckons they picked the b…',
  'created_at': '20220603-104325',
  'sentiment': 'neutral'},
 {'twitter_id': 1532674252956123136,
  'username': 'ArabiskaPartiet',
  'text': '"It’s not just that the 4 billion tons of uranium in seawater now would fuel a thousand 1000-MW nuclear power plant… https://t.co/WO4uvlNwy8',
  'created_at': '20220603-104312',
  'sentiment': 'neutral'}]

In [23]:
for i in tweets:
    print(tuple(i.values()))
    print(",".join(i.keys()))

(1532674306383159298, 'Luciano Pestarino', 'RT @giginator_: #Uranium names ripped overnight in Nth America\n$CCJ +7%\n$DNN +5%\n$URG +14%\n$UUUU +10%\nCantor Fitz reckons they picked the b…', '20220603-104325', 'neutral')
twitter_id,username,text,created_at,sentiment
(1532674252956123136, 'ArabiskaPartiet', '"It’s not just that the 4 billion tons of uranium in seawater now would fuel a thousand 1000-MW nuclear power plant… https://t.co/WO4uvlNwy8', '20220603-104312', 'neutral')
twitter_id,username,text,created_at,sentiment


In [36]:
from datetime import datetime, timedelta


In [37]:
(datetime.now()-timedelta(days=7)).strftime("%Y-%m-%d")

'2022-05-27'

In [25]:
conn.close()